In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [5]:
%sendtofile main.py

# Generate flashes on an ESP8266 to simulate the device
import machine
p = machine.Pin(2, machine.Pin.OUT)

import math, time

p.value(1)
th = 0
while True:
    x = math.sin(th)
    th += 0.2
    t = int((x+1.5)*1000)
    p.value(0)
    time.sleep_ms(100)
    p.value(1)
    time.sleep_ms(t)
    

Sent 16 lines (240 bytes) to main.py.


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [3]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
%sendtofile --source ../basicsockets/minimalmdns.py


Sent 86 lines (3060 bytes) to minimalmdns.py.


In [7]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [8]:
%sendtofile main.py

import machine, time, network

wifiname = "DoESLiverpool"
wifipassword = "decafbad00"
mqttbrokername = "mqtt.local"

pled = machine.Pin(2, machine.Pin.OUT)
pdetector = machine.Pin(15, machine.Pin.IN)

pled.value(1)
time.sleep(3.5)  # chance to interrupt before wifi is turned on

si = network.WLAN(network.STA_IF)
si.active(True)

print("Connecting to", wifiname)
si.connect(wifiname, wifipassword)
while not si.isconnected():
    time.sleep(0.1)
pled.value(0) 
print("connected to wifi")

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)
    
if mqttbrokername[-6:] == ".local":
    
    mqttbroker = mdnshostnametoipnumber(si, mqttbrokername)
    print("mDNS completed", mqttbrokername, mqttbroker)
else:
    mqttbroker = mqttbrokername
pled.value(1) 


from umqtt.robust import MQTTClient
client = MQTTClient("esp32powflash", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
pled.value(0) 
print("connected to mqtt")

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

n = 0
t0 = 0
while True:
    x = machine.time_pulse_us(pdetector, 0, 10000000)//1000
    if x < 0:
        continue
    if x < 15:
        continue
    t = time.ticks_ms()
    dt = t - t0
    if dt < 15:
        continue
    print(dt, x, n)
    try:
        client.publish(b"esp32powflash/ticks", "%d %d %d" % (dt, x, n))
    except Exception as e:
        print(e)
    t0 = t
    n += 1
    pled.value(n%2) 


Sent 70 lines (1533 bytes) to main.py.


In [77]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 54, in <module>
KeyboardInterrupt: 
MicroPython v1.10-331-ge38c68310-dirty on 2019-05-09; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [81]:
n = 0
t0 = 0
while True:
    x = machine.time_pulse_us(pdetector, 0, 10000000)//1000
    if x < 0:
        continue
    if x < 50:
        continue
    t = time.ticks_ms()
    dt = t - t0
    if dt < 50:
        continue
    print(dt, x, n)
    client.publish(b"esp32powflash/ticks", "%d %d %d" % (dt, x, n))
    t0 = t
    n += 1
    pled.value(n%2) 


115576 102 0
2460 102 1
.2543 102 2
2591 102 3
.2598 102 4
2566 102 5
.2495 102 6
2388 102 7
2251 102 8
.2087 102 9
1903 102 10
1708 102 11
.1509 102 12
1313 102 13
1127 102 14
963 102 15
821 102 16
713 102 17
639 102 18
.603 102 19
608 102 20
652 102 21
732 102 22

**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

